# Data Processing 

Hydrating a system is an essential part of a well-oiled recommendation system. As datasets grow larger and larger, it proved benificial to use specialized computing libraries to help handle the data at scale. 

## PySpark 

Python API access to the Spark computing library that can be used to help process and transform large-scale datasets 

PySpark provides a convenient SQL API that allows us to write what seems to be SQL queries against large datasets

## Simple PySpark Recommender System 

`SparkSession` serves as an entry point for all Spark functionalitites. 

You *must* start a session if you want to do any building/manipulating in Spark 

In [6]:
from pyspark.sql import SparkSession

# start a spark session 
spark = (
    SparkSession.builder
    .appName("simple-spark-recommender")
    .config("spark.memory.offHeap.enabled", "true") # 
    .config("spark.memory.offHeap.size", "10g")
    .getOrCreate()
)

24/06/24 09:23:52 WARN SparkContext: Another SparkContext is being constructed (or threw an exception in its constructor). This may indicate an error, since only one SparkContext should be running in this JVM (see SPARK-2243). The other SparkContext was created at:
org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.lang.reflect.Constructor.newInstance(Constructor.java:423)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
py4j.ClientServ

In [7]:
# create a dataframe 
df = spark.read.csv("../data/saved_track_features.csv", header=True, escape="\"")